## Pan Number analysis:


In [51]:
import pandas as pd
import re

In [52]:
df = pd.read_excel('/content/PAN Number Validation Dataset.xlsx')
df.head()

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L


In [53]:
total_records = len(df)
print('Total Records:', total_records)
#print('Total Records:', len(df))
print()
print(df.info())

Total Records: 10000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Pan_Numbers  9035 non-null   object
dtypes: object(1)
memory usage: 78.3+ KB
None


## Data Cleaning and preprocessing:

In [54]:
df['Pan_Numbers'] = df['Pan_Numbers'].astype('string').str.strip().str.upper()
df.head(7)

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L
5,HTJYM3835H
6,YQTAP6661X


In [55]:
df.isna().sum()

,0
Pan_Numbers,965


In [56]:
print('Initial records:', len(df))
print()
df = df.replace({'Pan_Numbers':''}, pd.NA).dropna(subset = 'Pan_Numbers')
print('After removing NAN records:', len(df))


Initial records: 10000

After removing NAN records: 9033


In [57]:
## Unique values in the data set:
print('Unique records in dataset:\n', df['Pan_Numbers'].nunique())
## check for duplicates:
print('Duplicates record in dataset:\n', df.duplicated().sum())
df.drop_duplicates(subset='Pan_Numbers', keep = 'first',inplace=True)
print('After removing duplicates:', len(df))

Unique records in dataset:
 9025
Duplicates record in dataset:
 8
After removing duplicates: 9025


## Data Validation:

In [58]:
# Adjacent Character (alphabets)cannot be the same (like AABBC)
def has_adjacent_repitition(pan): # AABCD
    # for i in range(len(pan)-1):
    #     if pan[i] == pan[i+1]:
    #         return True
    # return False
    return any(pan[i] == pan[i+1] for i in range(len(pan)-1))
print(has_adjacent_repitition('AABCD'))

True


In [59]:
# all five characters can't form a Sequential (like: ABCDE, BCDEF)
def is_sequential(pan):
    # for i in range(len(pan)-1):
    #     if ord(pan[i+1])- ord(pan[i]) != 1:
    #         return False
    # return True

    return all(ord(pan[i+1])- ord(pan[i]) == 1 for i in range(len(pan)-1))
print(is_sequential('ABCDE'))


True


In [60]:
def is_valid_pan(pan):
    if len(pan) != 10:
        return False

    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$', pan):
        return False

    if has_adjacent_repitition(pan):
        return False

    if is_sequential(pan):
        return False

    return True

## Categorisation:
> Categorize valud and invalid pan:

In [61]:
df['Status'] = df['Pan_Numbers'].apply(lambda x: 'Valid' if is_valid_pan(x) else 'Invalid')
df.head()

,Pan_Numbers,Status
0,VGLOD3180G,Valid
1,PHOXD7232L,Valid
2,MGEPH6532A,Valid
3,JJCHK4574O,Invalid
4,XTQIJ2330L,Invalid


## Data Summary Records

In [62]:
valid_count = (df['Status']=='Valid').sum()
invalid_count = (df['Status']=='Invalid').sum()
missing_count = total_records - (valid_count + invalid_count)
print('Total Records:', total_records)
print('Valid PANs:', valid_count)
print('Invalid PANs:', invalid_count)
print('Missing PANs:', missing_count)




Total Records: 10000
Valid PANs: 3193
Invalid PANs: 5832
Missing PANs: 975


In [64]:
df_summary = pd.DataFrame({'TOTAL PROCESSED RECORDS': [total_records],
                           'TOTAL VALID COUNT RECORDS': [valid_count],
                           'TOTAL INVALID COUNT RECORDS': [invalid_count],
                           'TOTAL MISSING RECORDS': [missing_count]
                           })
df_summary

,TOTAL PROCESSED RECORDS,TOTAL VALID COUNT RECORDS,TOTAL INVALID COUNT RECORDS,TOTAL MISSING RECORDS
0,10000,3193,5832,975


# Create a new excel file with two worksheet:

In [65]:
with pd.ExcelWriter('Pan Validation Result.xlsx') as writer:
    df.to_excel(writer, sheet_name= 'Pan Validation Result', index=False)
    df_summary.to_excel(writer, sheet_name= 'Summary', index=False)